In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import pack_vis_sol
import matplotlib.pyplot as plt
importlib.reload(kgs)
#pack_vis.plot_polygons([kgs.center_tree] + kgs.convex_breakdown)

In [ ]:
import numpy as np
import glob
for N_trees in [58,60,62,74,76,78,68]:
    files = glob.glob(kgs.temp_dir + f'/controller_runs/done/ga_N{N_trees}_*.pickle')
    #print(N_trees,files)
    if len(files)>0:
        plt.figure()
        for f in files:
            if ('seed90' in f) or ('seed92' in f) or ('seed91' in f):
                dat = kgs.dill_load(f)
                plt.plot([x[0] for x in dat.ga.best_costs_per_generation[0]], label=f[20:])
        plt.title(f'N_trees={N_trees}')
        plt.legend()
        plt.xlabel('Number of generations')
        plt.ylabel('Score (before legalize)')
        plt.grid()
        plt.pause(0.001)

In [ ]:
# Compare final scores across multiple seeds
SEEDS_TO_DO = [90, 91, 92]

import numpy as np
import glob
import matplotlib.pyplot as plt
import re

# Collect final scores for each seed
seed_scores = {seed: {} for seed in SEEDS_TO_DO}

for N_trees in np.arange(21,201,1):
    files = glob.glob(kgs.temp_dir + f'/*runs/done/ga_N{N_trees}_*.pickle')
    
    for f in files:
        # Extract seed from filename
        seed = None
        for s in SEEDS_TO_DO:
            if f'seed{s}' in f or f'_{s}' in f or f's{s}' in f:
                seed = s
                break
        
        if seed is None:
            # Try to parse seed from filename more generically
            seed_match = re.search(r'seed(\d+)|s(\d+)|_(\d+)(?!.*N\d)', f)
            if seed_match:
                parsed_seed = int(seed_match.group(1) or seed_match.group(2) or seed_match.group(3))
                if parsed_seed in SEEDS_TO_DO:
                    seed = parsed_seed
        
        if seed is not None:
            # Load data and get final score
            dat = kgs.dill_load(f)
            final_score = dat.ga.champions[0].phenotype.h[0,0].get()**2/N_trees  # Last generation's best cost
            seed_scores[seed][N_trees] = final_score

# Collect all N_trees values across all seeds
all_N_trees = set()
for seed in SEEDS_TO_DO:
    all_N_trees.update(seed_scores[seed].keys())
all_N_trees = sorted(all_N_trees)

if all_N_trees:
    # Create plot
    plt.figure(figsize=(10, 6))
    markers = ['o', 's', '^', 'D', 'v', '<', '>', 'p', '*', 'h']
    
    for i, seed in enumerate(SEEDS_TO_DO):
        # Get values for this seed (None for missing data)
        n_trees_list = []
        values = []
        for n in all_N_trees:
            if n in seed_scores[seed]:
                n_trees_list.append(n)
                values.append(seed_scores[seed][n])
        
        if values:  # Only plot if there's at least one value
            marker = markers[i % len(markers)]
            plt.plot(n_trees_list, values, marker=marker, linestyle='-', 
                    label=f'Seed {seed}, sum {np.sum(values):.4f} ({len(values)} points)', 
                    linewidth=2, markersize=8)
    
    plt.xlabel('Number of Trees', fontsize=12)
    plt.ylabel('Final Score', fontsize=12)
    plt.title('Final Score vs Number of Trees (Seed Comparison)', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print(f"Plotted {len(all_N_trees)} total N_trees values: {all_N_trees}")
    for seed in SEEDS_TO_DO:
        print(f"  Seed {seed}: {len(seed_scores[seed])} data points")
else:
    print("No N_trees values found")

In [ ]:
for N_trees in [156]+list(np.arange(90,41,-2)):
    seeds = [90,91,92]
    res = []
    for s in seeds:
        f = kgs.temp_dir + f'/controller_runs/done/ga_N{N_trees}_seed{s}_done.pickle'
        try:
            dat = kgs.dill_load(f)
            res.append(dat.ga.champions[0])
        except Exception as e:
            res.append(None)
    _,ax = plt.subplots(1,3, figsize=(18,6))
    for i,r in enumerate(res):
        if not r is None:
            plt.sca(ax[i])
            pack_vis_sol.pack_vis_sol(r.phenotype, ax=ax[i])
            plt.title(f'Seed {seeds[i]}, score {r.phenotype.h[0,0].get()**2/N_trees:.8f}')
    plt.suptitle(f'N_trees={N_trees}')
    plt.pause(0.001)

In [ ]:
dat = kgs.dill_load(kgs.temp_dir + f'/controller_runs/done/ga_N25_seed87_done.pickle')
dat.ga.diagnostic_plots(-1,None)

In [ ]:
2